In [30]:
import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [48]:
def load(month):
    string = f"{month}"
    print(string)
    
def extract(*month):
    print(month)
    for m in month:
        print(m)

In [49]:
month = '05'
def main(*month):
    print(month)
    extract(*month)
    load(*month)

In [50]:
main(month)

('05',)
('05',)
05
05


In [56]:
month = '05', '06'
def test(*month):
    print(*month)
test(month)

('05', '06')


# Extract Data

In [2]:
def extract_data_sales_offline(year: str, *months):
    # Data Existing
    df_offline = pd.DataFrame()

    for month in months:
        data_offline = pd.read_parquet(
            f"s3://mega-dev-lake/ProcessedData/Sales/sales_detail_indie/{year}/{month}/data.parquet"
        )

        data_offline = data_offline[
            (data_offline["status_order"] != 7)
            & (data_offline["issettled"] != False)
            & (data_offline["istransaction"] != "0")
            & (data_offline["channel"] != "ONLINE")
        ]

        df_offline = pd.concat([df_offline, data_offline])

    df_offline["order_create_date"] = pd.to_datetime(
        df_offline["order_create_date"]
    ).dt.strftime("%Y-%m-%d")
    return df_offline


def extract_data_sales_online(year: str, *months):
    df_online = pd.DataFrame()

    for month in months:
        data_online = pd.read_parquet(
            f"s3://mega-dev-lake/ProcessedData/Sales/sales_online/{year}/{month}/data.parquet"
        )

        df_online = pd.concat([df_online, data_online])

    df_online.columns = df_online.columns.str.lower()
    df_online.columns = df_online.columns.str.replace(" ", "_")
    df_online = df_online[df_online["status_sales"] != "CANCELED"]
    df_online["date"] = pd.to_datetime(df_online["date"]).dt.strftime("%Y-%m-%d")

    df_online["marketplace"] = np.where(
        df_online["marketplace"].isin(
            [
                "MANZONESTORE.ID",
                "MINIMALSTORE.ID",
                "MOCSTORE.ID",
                "WEBSITE",
            ]
        ),
        "SHOPIFY",
        df_online["marketplace"],
    )
    
    return df_online


def extract_pattern_data(pattern_path, sheet_name, *months):
    # Data Pattern
    pattern = pd.read_excel(pattern_path, sheet_name=sheet_name)
    list_month = [month for month in months]

    pattern = pattern[
        pd.to_datetime(pattern["pattern_date"]).dt.strftime("%m").isin(list_month)
    ]
    pattern["pattern_date"] = pd.to_datetime(pattern["pattern_date"]).dt.strftime(
        "%Y-%m-%d"
    )
    pattern["pattern_last_year"] = pd.to_datetime(
        pattern["pattern_last_year"]
    ).dt.strftime("%Y-%m-%d")
    return pattern


def extract_master_store(*months: str):
    master = pd.read_parquet(
        "s3://mega-dev-lake/Staging/Master/Master Store/2024/data.parquet"
    )

    month_name = [pd.to_datetime(month, format="%m").month_name() for month in months]

    if month_name != "All":
        master = master[master["month"].isin(month_name)]
    else:
        pass

    return master


def extract_master_online(master):
    master_online = master[master["main_channel"] == "ONLINE"]
    master_online["marketplace"] = master_online["stdname"].str.split(" ").str.get(2)
    master_online["brand"] = master_online["stdname"].str.split(" ").str.get(1)
    master_online["marketplace"] = master_online["marketplace"].fillna("")

    master_online["marketplace"] = np.where(
        master_online["marketplace"].isin(
            [
                "MANZONESTORE.ID",
                "MINIMALSTORE.ID",
                "MOCSTORE.ID",
                "WEBSITE",
            ]
        ),
        "SHOPIFY",
        master_online["marketplace"],
    )

    master_online = master_online[master_online["openstatus"] == "OPEN"]
    return master_online

def main_extract(year, *months):
    pattern_path = "s3://mega-lake/ProcessedData/Sales/pattern/pattern_24_with_festive.xlsx"
    
    sheet_name = "offline"

    # Extract existing data
    df_offline = extract_data_sales_offline(year, *months)
    df_online = extract_data_sales_online(year, *months)
    pattern = extract_pattern_data(pattern_path, sheet_name, *months)
    master_store = extract_master_store(*months)
    master_online = extract_master_online(master_store)
    
    # Extract historical data (last year)
    list_month_lastyear = pd.to_datetime(pattern[~(pattern['pattern_last_year'].isna())]['pattern_last_year']).dt.strftime("%m").unique()
    year_hist = pd.to_datetime(pattern[~(pattern['pattern_last_year'].isna())]['pattern_last_year']).dt.strftime("%Y").unique()[0]
    
    df_offline_hist = extract_data_sales_offline(year_hist, *list_month_lastyear)
    df_online_hist = extract_data_sales_online(year_hist, *list_month_lastyear)
    
    return df_offline, df_online, pattern, master_store, master_online, df_offline_hist, df_online_hist

# Transformasi

In [4]:
def transformasi_sales(df_offline, df_online, master_online):
    df_offline_agg = (
        df_offline.groupby(
            ["dataareaid", "axcode", "order_create_date", "category", "subcategory"],
            as_index=False,
        )
        .agg({"cust_paid_peritem": "sum", "quantity": "sum", "hpp_total": "sum"})
        .rename(columns={"order_create_date": "date"})
    )

    df_online_agg = (
        master_online[["dataareaid", "axcode", "brand", "marketplace"]]
        .merge(
            df_online.groupby(
                ["brand", "marketplace", "date", "category", "subcategory"],
                as_index=False,
            )
            .agg(
                {"value_after_voucher": "sum", "qty_sold": "sum", "existing_hpp": "sum"}
            )
            .rename(
                columns={
                    "value_after_voucher": "cust_paid_peritem",
                    "qty_sold": "quantity",
                    "existing_hpp": "hpp_total",
                }
            ),
            "right",
            on=["brand", "marketplace"],
        )
        .drop(columns=["brand", "marketplace"])
    )

    df_agg = pd.concat([df_offline_agg, df_online_agg])
    return df_agg


def transformasi_with_pattern_exsiting(df_agg, pattern):
    df_agg_pattern = df_agg.merge(
        pattern[["pattern_date", "pattern_last_year"]],
        "left",
        left_on="date",
        right_on="pattern_date",
    )
    return df_agg_pattern


def transformasi_with_pattern_hist(df_agg, pattern):
    df_agg_pattern = df_agg.merge(
        pattern[["pattern_date", "pattern_last_year"]],
        "left",
        left_on="date",
        right_on="pattern_last_year",
    )
    return df_agg_pattern


def transformasi_growth_daily(df_existing, df_hist):
    # Transformasi Growth
    df_hist["date"] = df_hist["pattern_date"]
    df_growth = df_existing.merge(
        df_hist,
        "outer",
        on=[
            "dataareaid",
            "axcode",
            "category",
            "subcategory",
            "date",
            "pattern_last_year",
        ],
        suffixes=("_this_year", "_last_year"),
        indicator=True,
    ).reset_index(drop=True)
    df_growth[df_growth.select_dtypes(include="number").columns] = df_growth[
        df_growth.select_dtypes(include="number").columns
    ].fillna(0)

    return df_growth


def transformasi_to_report(df_growth, master_store):
    df_report = master_store[
        [
            "dataareaid",
            "channel",
            "brand",
            "axcode",
            "stdname",
            "so dept head",
            "area head",
            "city head",
        ]
    ].merge(df_growth, "right", on=["axcode", "dataareaid"]).drop(columns=['pattern_date_this_year', 'pattern_date_last_year', '_merge'])


    df_report.columns = df_report.columns.str.replace("_", " ").str.title()
    return df_report

def main_transformasi(df_offline, df_online, pattern, df_offline_hist, df_online_hist, master_online, master_store):
    df_sales_existing = transformasi_sales(df_offline, df_online, master_online)
    df_sales_hist = transformasi_sales(df_offline_hist, df_online_hist, master_online)
    
    df_sales_pattern_existing = transformasi_with_pattern_exsiting(df_sales_existing, pattern)
    df_sales_pattern_hist = transformasi_with_pattern_hist(df_sales_hist, pattern)

    max_date = df_sales_pattern_existing['date'].max()
    df_sales_pattern_hist = df_sales_pattern_hist[df_sales_pattern_hist['pattern_date'] <= max_date]

    df_growth = transformasi_growth_daily(df_sales_pattern_existing, df_sales_pattern_hist).query("_merge == 'both'")
    df_report = transformasi_to_report(df_growth,master_store)
    
    return df_report

In [5]:
def load(df, month):
    file_name = "data.xlsx"
    path = f"s3://mega-dev-lake/Staging/Sales/akumulasi/growth_peritem_v2/{month}/{file_name}"
    df.to_excel(path, index=False)

In [6]:
def main(year, *month):
    print("--- Start Extract ---")
    df_offline, df_online, pattern, master_store, master_online, df_offline_hist, df_online_hist = main_extract(year,*month)
    
    print("--- Start Transformasi ---")
    df_report = main_transformasi(df_offline, df_online, pattern, df_offline_hist, df_online_hist, master_online, master_store)
    
    print("--- Start Load Data ---")
    load(df_report)
    print("--- Data Loaded ---")

In [7]:
# Data Existing
df_offline = pd.DataFrame()
df_online = pd.DataFrame()

for month in ['05']:
    data_offline = getSalesDetail("2024", month)
    data_online = getSalesDetailOnline("2024", month)
    
    df_offline= pd.concat([df_offline, data_offline])
    df_online= pd.concat([df_online, data_online])
    
df_offline['order_create_date'] = pd.to_datetime(df_offline['order_create_date']).dt.strftime("%Y-%m-%d")
df_online['date'] = pd.to_datetime(df_online['date']).dt.strftime("%Y-%m-%d")

NameError: name 'getSalesDetail' is not defined

In [ ]:
# Data Pattern
pattern = pd.read_excel(
    "s3://mega-lake/ProcessedData/Sales/pattern/pattern_24_with_festive.xlsx",
    sheet_name="offline",
)

pattern = pattern[pd.to_datetime(pattern['pattern_date']).dt.strftime("%m").isin(['05'])]
pattern['pattern_date'] = pd.to_datetime(pattern['pattern_date']).dt.strftime("%Y-%m-%d")
pattern['pattern_last_year'] = pd.to_datetime(pattern['pattern_last_year']).dt.strftime("%Y-%m-%d")

In [44]:
list_month_lastyear = pd.to_datetime(pattern[~(pattern['pattern_last_year'].isna())]['pattern_last_year']).dt.strftime("%m").unique()
list_year_lastyear = pd.to_datetime(pattern[~(pattern['pattern_last_year'].isna())]['pattern_last_year']).dt.strftime("%Y").unique()

In [5]:
# Data Last Year by Pattern
df_last_year = pd.DataFrame()
df_online_last_year = pd.DataFrame()
for month in list_month_lastyear:
    offline = getSalesDetail("2023", month)
    online = getSalesDetailOnline("2023", month)
    df_last_year = pd.concat([df_last_year, offline])
    df_online_last_year = pd.concat([df_online_last_year, online])
    
df_last_year['order_create_date'] = pd.to_datetime(df_last_year['order_create_date']).dt.strftime("%Y-%m-%d")
df_online_last_year['date'] = pd.to_datetime(df_online_last_year['date']).dt.strftime("%Y-%m-%d")

In [8]:
# Data Master
master = getMasterOnline("May")

# Transformasi

In [6]:
# Transformasi data existing

df_offline_agg = (
    df_offline.groupby(
        ["dataareaid", "axcode", "order_create_date", "category", "subcategory"],
        as_index=False,
    )
    .agg({"cust_paid_peritem": "sum", "quantity": "sum", "hpp_total": "sum"})
    .rename(columns={"order_create_date": "date"})
)

df_online_agg = (
    master[["dataareaid", "axcode", "brand", "marketplace"]]
    .merge(
        df_online.groupby(
            ["brand", "marketplace", "date", "category", "subcategory"], as_index=False
        )
        .agg({"value_after_voucher": "sum", "qty_sold": "sum", "existing_hpp": "sum"})
        .rename(
            columns={
                "value_after_voucher": "cust_paid_peritem",
                "qty_sold": "quantity",
                "existing_hpp": "hpp_total",
            }
        ),
        "right",
        on=["brand", "marketplace"],
    )
    .drop(columns=["brand", "marketplace"])
)

df_24 = pd.concat([df_offline_agg, df_online_agg])

In [14]:
df_24_pattern = df_24.merge(
    pattern[["pattern_date", "pattern_last_year"]],
    "left",
    left_on="date",
    right_on="pattern_date",
)

In [13]:
# Transformasi Data Last Year
df_last_year = df_last_year[
    (df_last_year["status_order"] != 7)
    & (df_last_year["issettled"] != False)
    & (df_last_year["istransaction"] != "0")
    & (df_last_year["channel"] != "ONLINE")
]
df_online_last_year = df_online_last_year[
    df_online_last_year["status_sales"] != "CANCELED"
]

df_last_year_agg = (
    df_last_year.groupby(
        ["dataareaid", "axcode", "order_create_date", "category", "subcategory"],
        as_index=False,
    )
    .agg({"cust_paid_peritem": "sum", "quantity": "sum", "hpp_total": "sum"})
    .rename(columns={"order_create_date": "date"})
)

df_online_last_year["marketplace"] = np.where(
    df_online_last_year["marketplace"].isin(
        ["MANZONESTORE.ID", "MOCSTORE.ID", "MINIMALSTORE.ID", "WEBSITE"]
    ),
    "SHOPIFY",
    df_online_last_year["marketplace"],
)

df_online_last_year_agg = (
    master[["dataareaid", "axcode", "brand", "marketplace"]]
    .merge(
        df_online_last_year.groupby(
            ["brand", "marketplace", "date", "category", "subcategory"], as_index=False
        )
        .agg({"value_after_voucher": "sum", "qty_sold": "sum", "existing_hpp": "sum"})
        .rename(
            columns={
                "value_after_voucher": "cust_paid_peritem",
                "qty_sold": "quantity",
                "existing_hpp": "hpp_total",
            }
        ),
        "right",
        on=["brand", "marketplace"],
    )
    .drop(columns=["brand", "marketplace"])
)

df_23 = pd.concat([df_last_year_agg, df_online_last_year_agg])


In [15]:
max_date = df_24_pattern['date'].max()

df_23_pattern = df_23.merge(
    pattern[["pattern_date", "pattern_last_year"]],
    "left",
    left_on="date",
    right_on="pattern_last_year",
)
df_23_pattern["date"] = df_23_pattern["pattern_date"]
df_23_pattern = df_23_pattern[df_23_pattern['date'] <= max_date]

In [18]:
# Transformasi Growth
df_growth = df_24_pattern.merge(
    df_23_pattern,
    "outer",
    on=[
        "dataareaid",
        "axcode",
        "category",
        "subcategory",
        "date",
        "pattern_last_year",
    ],
    suffixes=("_this_year", "_last_year"),
).reset_index()

max_date = pd.to_datetime(df_24["date"]).max()

df_growth = df_growth[pd.to_datetime(df_growth["date"]) <= max_date]
df_growth[df_growth.select_dtypes(include='number').columns] = df_growth[df_growth.select_dtypes(include='number').columns].fillna(0)

In [24]:
# Transformasi Growth by Month
df_growth_monthly = df_24.merge(
    df_23,
    "outer",
    on=[
        "dataareaid",
        "axcode",
        "category",
        "subcategory",
    ],
    suffixes=("_this_year", "_last_year"),
).reset_index()

df_growth[df_growth.select_dtypes(include='number').columns] = df_growth[df_growth.select_dtypes(include='number').columns].fillna(0)

# max_date = pd.to_datetime(df_24["date"]).max()

In [25]:
df_growth.agg(
    {
        "cust_paid_peritem_this_year": "sum",
        "cust_paid_peritem_last_year": "sum",
        "quantity_this_year": "sum",
        "quantity_last_year": "sum",
    }
)

cust_paid_peritem_this_year    4.336349e+10
cust_paid_peritem_last_year    4.550924e+10
quantity_this_year             2.490620e+05
quantity_last_year             2.395240e+05
dtype: float64

# Validasi

In [31]:
df_23[df_23["date"].isin(df_growth["pattern_last_year"].unique())]['cust_paid_peritem'].sum()

45509237045.47619

In [25]:
master_store = getMasterStore("April")

In [26]:
df_growth = master_store[
    [
        "dataareaid",
        "channel",
        "brand",
        "axcode",
        "stdname",
        "so dept head",
        "area head",
        "city head",
    ]
].merge(df_growth, "right", on=["axcode", "dataareaid"])
df_growth.columns = df_growth.columns.str.replace("_", " ").str.title()

In [27]:
df_growth.columns = df_growth.columns.str.replace("_", " ").str.title()

In [28]:
df_growth.agg(
    {
        "Cust Paid Peritem This Year": "sum",
        "Cust Paid Peritem Last Year": "sum",
        "Quantity This Year": "sum",
        "Quantity Last Year": "sum",
    }
)

Cust Paid Peritem This Year    2.813675e+11
Cust Paid Peritem Last Year    2.746963e+11
Quantity This Year             1.579866e+06
Quantity Last Year             1.437285e+06
dtype: float64

In [29]:
df_24.agg({"cust_paid_peritem": "sum"})

cust_paid_peritem    2.813675e+11
dtype: float64

In [30]:
df_23.agg({"cust_paid_peritem": "sum"})

cust_paid_peritem    2.746963e+11
dtype: float64

In [32]:
df_growth.to_excel("Data Sales Growth Per Cat Periode Mar - April.xlsx", index=False)